# Import

In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, searches, regularizers, evaluators, trainers, datasets, distributions

# Dataset

In [2]:
dataset = datasets.ML100k()
n_user = dataset.n_user
n_item = dataset.n_item
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.LongTensor(test_set).to(device)

# Evaluator

In [3]:
# k
ks = [5, 10, 50]

recall = evaluators.RecallEvaluator(test_set, ks)
unpopularity = evaluators.UnpopularityEvaluator(test_set, ks)
f1_score = evaluators.F1ScoreEvaluator(test_set, ks)
# coverage = evaluators.CoverageEvaluator(test_set, ks)
# hubness = evaluators.HubnessEvaluator(test_set, ks)

# Sampler

In [4]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

# Model

In [5]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 50
no_progressbar = True

model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler, no_progressbar)

# Search

In [6]:
# Hyperparameters
search_range = 100

knn = searches.NearestNeighborhood(model)
mp = searches.Mymp(model, search_range)

# Training

In [7]:
# only traing no validation
trainer.fit(n_batch, n_epoch)

# Result

## Recall

In [8]:
trainer.valid(knn, recall)
trainer.valid_scores

,Recall@5,Recall@10,Recall@50
0,0.39998,0.57354,0.888818


In [9]:
trainer.valid(mp, recall)
trainer.valid_scores

,Recall@5,Recall@10,Recall@50
0,0.346535,0.505103,0.873143


## Unpopularity

In [10]:
trainer.valid(knn, unpopularity)
trainer.valid_scores

,Unpopularity@5,Unpopularity@10,Unpopularity@50
0,0.01368,0.049366,0.438712


In [11]:
trainer.valid(mp, unpopularity)
trainer.valid_scores

,Unpopularity@5,Unpopularity@10,Unpopularity@50
0,0.029902,0.079254,0.463608


## F1 score

In [12]:
trainer.valid(knn, f1_score)
trainer.valid_scores

,F1-score@5,F1-score@10,F1-score@50
0,0.02424,0.081883,0.514219


In [13]:
trainer.valid(mp, f1_score)
trainer.valid_scores

,F1-score@5,F1-score@10,F1-score@50
0,0.048188,0.12425,0.541927
